In [1]:
!pip install transformers[sentencepiece] datasets

In [ ]:
# Load datasets
from datasets import Dataset
import json
import gdown

url = 'https://drive.google.com/uc?id=1sAgDtEj-UjJECfTF6xfiWFk7lrTX7yoV'
filename = "articles_1000.json"
gdown.download(url, filename, quiet=False)
with open(filename, 'r') as f:
    data = json.load(f)
dataset1000 = Dataset.from_dict(data)
dataset300 = dataset1000.select(range(300))
dataset500 = dataset1000.select(range(500))

In [2]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformer_compressor import TransformerCompressor
from transformers import PreTrainedTokenizerFast
import numpy as np
import time
import gzip
from utils import batched_encode, batched_binarize



In [3]:
# Transformer-based compression

def transformer_compress(compressor: TransformerCompressor, tokenizer: PreTrainedTokenizerFast, dataset, batch_size, slice_len):
    print(type(slice_len))
    tick = time.time()
    data_by_buckets, encodings_by_buckets = batched_encode(compressor, tokenizer, dataset, batch_size, slice_len)
    binarized_by_bucket = batched_binarize(data_by_buckets, encodings_by_buckets)
    tock = time.time()
    transformer_time = tock - tick
    return transformer_time, data_by_buckets, binarized_by_bucket

# Gzip-based compression
def gzip_compress(data_by_buckets):
    tick = time.time()
    gzipped_by_bucket = {}
    for bucket in sorted(data_by_buckets.keys()):
        msgs = data_by_buckets[bucket]['article']
        gzipped = []
        for msg in msgs:
            res = gzip.compress(bytes(msg, encoding='utf-8'), compresslevel=9)
            gzipped.append(res)
        gzipped_by_bucket[bucket] = gzipped
    tock = time.time()
    gzip_time = tock - tick
    return gzip_time, gzipped_by_bucket

# Evaluation

def evaluate(compressor: TransformerCompressor, tokenizer: PreTrainedTokenizerFast, dataset, batch_size, slice_len):
    
    transformer_time, data_by_buckets, binarized_by_bucket = transformer_compress(compressor, tokenizer, dataset, batch_size, slice_len)
    gzip_time, gzipped_by_bucket = gzip_compress(data_by_buckets)


    og = []
    gz = []
    trans = []

    compression_stats_by_bucket = {}
    for bucket in sorted(data_by_buckets.keys()):
        og = og + list(map(lambda msg: len(bytes(msg, encoding='utf-8')), data_by_buckets[bucket]['article']))
        gz = gz + list(map(len, gzipped_by_bucket[bucket]))
        trans = trans + list(map(len, binarized_by_bucket[bucket]))
    return {'dataset_size': len(dataset), 'batch_size': batch_size, 'slice_len': slice_len, 
            'trans_time': transformer_time, 
            'gz_time': gzip_time, 'sizes': {'og': og, 'gz': gz, 'trans': trans}}



In [4]:
dataset100 = dataset1000.select(range(100))
# dataset20 = dataset1000.select(range(20))

# tiny_subset = dataset20.map(lambda elem: {'tiny_article': elem['article'][0:100]})
# tiny_subset = tiny_subset.remove_columns('article')
# tiny_subset = tiny_subset.add_column('article', tiny_subset['tiny_article']) 
# tiny_subset = tiny_subset.remove_columns('tiny_article')
# tiny_20 = tiny_subset
# tiny_10 = tiny_subset.select(range(10))

In [5]:
from os.path import exists


path = '../output/n{}_batch{}_slice{}.json'


def do_experiment(compressor, tokenizer, dataset, batch_size, slice_len, errors):
  print(f'DatasetSize {len(dataset)} | BatchSize: {batch_size}| SliceLen: {slice_len}')

  file_path = path.format(len(dataset), batch_size, slice_len)
  # Skip if experiment already done or caused error
  if f'n{len(dataset)}_batch{batch_size}_slice{slice_len}' in errors:
    print('Previously errored. Skipping...')
  if exists(file_path):
    print('Skipping...')
    return

  results = evaluate(compressor, tokenizer, dataset, batch_size, slice_len)
  with open(file_path, 'w') as f:
    json.dump(results, f)



In [ ]:
from os import mkdir
try:
  mkdir('../output')
except FileExistsError:
  pass

# Distilgpt2 is twice as fast and has comparable compression performance
# Bart, BigBird, Reformer and XLNet all have far worst performance
model = GPT2LMHeadModel.from_pretrained('distilgpt2').to('cuda')
tokenizer = GPT2TokenizerFast.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token
VOCAB_SIZE = tokenizer.vocab_size
PAD_TOKEN = tokenizer.pad_token_id
EOS_TOKEN = tokenizer.eos_token_id
compressor = TransformerCompressor(model, VOCAB_SIZE, PAD_TOKEN, EOS_TOKEN)

batch_size = 32
slices_lengths = [32,64,128,256,512]

errors = []

# TODO: Switch back after finish experiments
for slice_len in [256,512]:
    batch_size = 4
    try:
        do_experiment(compressor, tokenizer, dataset1000, batch_size, slice_len, errors)
    except RuntimeError: #Out of memory
        print(f'n{len(dataset)}_batch{batch_size}_slice{slice_len} failed')
        errors.append(f'n{len(dataset)}_batch{batch_size}_slice{slice_len}')


print(errors)

DatasetSize 1000 | BatchSize: 4| SliceLen: 256
<class 'int'>


0ex [00:00, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1147 > 1024). Running this sequence through the model will result in indexing errors


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

Buckets: [512, 1024, 1536, 2048, 2560, 3584]
Starting compression of bucket 512


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83/83 [13:37<00:00,  9.85s/it]


Starting compression of bucket 1024


 15%|███████████████████████████████▍                                                                                                                                                                             | 15/98 [04:55<27:15, 19.70s/it]

In [ ]:
import shutil
shutil.make_archive('outputs.zip', 'zip', '../output')

In [7]:
errors


[]